In [ ]:
# default_exp dataloaders

# Dataloaders

> Pytorch datasets, dataloaders, collate functions and vocabularies

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from mrl.imports import *
from mrl.torch_imports import *

In [ ]:
# export

SMILES_CHAR_VOCAB = ['#', '(', ')', '+', '-', '/', '0',
                 '1', '2', '3', '4', '5', '6', '7',
                 '8', '=', '@', 'B', 'C', 'F', 'H',
                 'I', 'N', 'O', 'P', 'S', '[', '\\',
                 ']', 'c', 'i', 'l', 'n', 'o', 'r', 's',
                 '*', ':']


SPECIAL_TOKENS = ['bos', 'eos', 'pad', 'unk']

MAPPING_TOKENS = ['[1*:1]', '[2*:1]', '[1*:2]', '[2*:2]', '[1*:3]',
                  '[2*:3]', '[1*:4]', '[2*:4]', '[1*:5]', '[2*:5]']

HALOGEN_REPLACE = {'Br':'R',
                   'Cl':'L'}

MAPPING_REPLACE = {'[1*:1]':'A',
                   '[2*:1]':'D',
                   '[1*:2]':'E',
                   '[2*:2]':'G',
                   '[1*:3]':'J',
                   '[2*:3]':'M',
                   '[1*:4]':'Q',
                   '[2*:4]':'T',
                   '[1*:5]':'U', 
                   '[2*:5]':'V'}

SMILE_REGEX = """(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|H|\(|\)|\.|=|
                 #|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"""

MAPPING_REGEX = """(\[.\*:.]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|H|\[|\]|\(|\)|\.|=|
                    #|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"""


In [ ]:
# export

def tokenize_by_character(smile):
    return [i for i in smile]

def tokenize_with_replacements(smile, replacement_dict):
    for k,v in replacement_dict.items():
        smile = smile.replace(k,v)
    return [i for i in smile]

def regex_tokenize(smile, regex):
    tokens = [token for token in regex.findall(smile)]
    return tokens

In [ ]:
# export

class Vocab():
    def __init__(self, itos):
        self.special_tokens = ['bos', 'eos', 'pad', 'unk']
        
        self.itos = self.special_tokens + [i for i in itos if not i in self.special_tokens]
        self.stoi = {self.itos[i]:i for i in range(len(self.itos))}
        self.unks = []
        
    def tokenize(self, input):
        raise NotImplementedError
        
    def numericalize(self, input):
        output = []
        for tok in input:
            if tok in self.stoi.keys():
                output.append(self.stoi[tok])
            else:
                output.append(self.stoi['unk'])
                self.unks.append(tok)
        return output
    
    def reconstruct(self, input):
        
        output = []
        for item in input:
            item = self.itos[item]
            if item=='eos':
                break
                
            if not item=='bos':
                output.append(item)
        
        return ''.join(output)
                
    def update_vocab(self):
        unks = list(set(self.unks))
        self.itos += unks
        self.stoi = {self.itos[i]:i for i in range(len(self.itos))}
        self.unks = []
        
    def update_vocab_from_data(self, inputs):
        _ = [self.numericalize(self.tokenize(i)) for i in inputs]
        self.update_vocab()
        
        
class CharacterVocab(Vocab):
    def tokenize(self, smile):
        toks = tokenize_by_character(smile)
        toks = ['bos'] + toks + ['eos']
        return toks
    
    
class CharacterReplaceVocab(Vocab):
    def __init__(self, itos, replace_dict):
        self.replace_dict = replace_dict
        self.reverse_dict = {v:k for k,v in replace_dict.items()}
        for rep in self.reverse_dict.keys():
            if not rep in itos:
                itos.append(rep)
        super().__init__(itos)
        
    def tokenize(self, smile):
        toks = tokenize_with_replacements(smile, self.replace_dict)
        toks = ['bos'] + toks + ['eos']
        return toks
    
    def reconstruct(self, input):
        output = []
        for item in input:
            item = self.itos[item]
            if item=='eos':
                break
            
            if not item=='bos':
                if item in self.reverse_dict.keys():
                    item = self.reverse_dict[item]

                output.append(item)
        
        return ''.join(output)
    
    
class RegexVocab(Vocab):
    def __init__(self, itos, pattern):
        super().__init__(itos)
        
        self.pattern = pattern
        self.regex = re.compile(self.pattern)
        
    def tokenize(self, smile):
        toks = regex_tokenize(smile, self.regex)
        toks = ['bos'] + toks + ['eos']
        return toks

In [ ]:
# export

def test_reconstruction(vocab, inputs):
    fails = []
    for item in inputs:
        recon = vocab.reconstruct(vocab.numericalize(vocab.tokenize(item)))
        if not item==recon:
            fails.append((item, recon))
            
    return fails

In [ ]:
# export

def batch_sequences(sequences, pad_idx):
    
    max_len = max([len(i) for i in sequences])+1
    bs = len(sequences)
    
    batch_tensor = torch.zeros((bs, max_len)).long() + pad_idx
    
    for i,item in enumerate(sequences):
        batch_tensor[i,:item.shape[0]] = item
        
    return batch_tensor
    
    
def lm_collate(batch, pad_idx, batch_first=True):
    
    batch_tensor = batch_sequences(batch, pad_idx)
        
    if batch_first:
        output = (batch_tensor[:,:-1], batch_tensor[:,1:])
    else:
        batch_tensor = batch_tensor.T
        output = (batch_tensor[:-1,:], batch_tensor[1:,:])
        
    return output

def sequence_prediction_collate(batch, pad_idx, batch_first=True):
    
    batch_tensor = batch_sequences([i[0] for i in batch], pad_idx)
    y_vals = torch.stack([i[1] for i in batch])
    y_vals = y_vals.squeeze(-1)

    if not batch_first:
        batch_tensor = batch_tensor.T
        
    return (batch_tensor, y_vals)

def fp_collate(batch):
    fps = torch.stack([i[0] for i in batch])
    y_vals = torch.stack([i[1] for i in batch])
    y_vals = y_vals.squeeze(-1)
    return (fps, y_vals)

In [ ]:
# export

class BaseDataset(Dataset):
    def __init__(self, collate_function):
        self.collate_function = collate_function
        
    def dataloader(self, bs, num_workers=-1, **dl_kwargs):
        if num_workers==-1:
            num_workers=os.cpu_count()
        return DataLoader(self, batch_size=bs, num_workers=num_workers, 
                          collate_fn=self.collate_function, **dl_kwargs)

class TextDataset(BaseDataset):
    def __init__(self, smiles, vocab, collate_function=None):
        self.smiles = smiles
        self.vocab = vocab
        if collate_function is None:
            collate_function = partial(lm_collate, pad_idx=self.vocab.stoi['pad'])
        
        super().__init__(collate_function)
        
    def __len__(self):
        return len(self.smiles)
    
    def __getitem__(self, idx):
        smile = self.smiles[idx]
        tokens = self.vocab.tokenize(smile)
        ints = self.vocab.numericalize(tokens)
        ints = torch.LongTensor(ints)
        return ints
    
        
class TextPredictionDataset(TextDataset):
    def __init__(self, smiles, y_vals, vocab, collate_function=None):
        
        if collate_function is None:
            collate_function = partial(sequence_prediction_collate, pad_idx=vocab.stoi['pad'])
        
        super().__init__(smiles, vocab, collate_function)
        
        self.y_vals = y_vals
        
    def __getitem__(self, idx):
        ints = super().__getitem__(idx)
        y_val = torch.Tensor([self.y_vals[idx]]).float()
        return (ints, y_val)

In [ ]:
# export

class FPDataset(BaseDataset):
    def __init__(self, smiles, y_vals, fp_function, collate_function=None):
        if collate_function is None:
            collate_function = fp_collate
        super().__init__(collate_function)
        
        self.smiles = smiles
        self.y_vals = y_vals
        self.fp_function = fp_function
        
    def __len__(self):
        return len(self.smiles)
    
    def __getitem__(self, idx):
        smile = self.smiles[idx]
        fp = self.fp_function(smile)
        fp = torch.FloatTensor(fp)
        y_val = torch.FloatTensor([self.y_vals[idx]])
        return (fp, y_val)

In [ ]:
df = pd.read_csv('files/smiles.csv')

In [ ]:
vocab = CharacterVocab(SMILES_CHAR_VOCAB)

In [ ]:
ds = TextDataset(df.smiles.values, vocab)

In [ ]:
dl = ds.dataloader(32)

In [ ]:
x,y = next(iter(dl))

In [ ]:
x.shape

torch.Size([32, 66])

In [ ]:
y.shape

torch.Size([32, 66])

In [ ]:
ds = TextPredictionDataset(df.smiles.values, np.random.randn(df.smiles.values.shape[0]),
                          vocab)

In [ ]:
dl = ds.dataloader(32)

In [ ]:
x,y = next(iter(dl))

In [ ]:
x.shape

torch.Size([32, 67])

In [ ]:
y.shape

torch.Size([32])

In [ ]:
from mrl.chem import *

//anaconda3/envs/insight/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.
  return f(*args, **kwds)


In [ ]:
ds = FPDataset(df.smiles.values, np.random.randn(df.smiles.values.shape[0]), ECFP6)

In [ ]:
dl = ds.dataloader(32)

In [ ]:
x,y = next(iter(dl))

In [ ]:
x.shape

torch.Size([32, 2048])

In [ ]:
y.shape

torch.Size([32])

In [ ]:
x[0].sum()

tensor(73.)

In [ ]:
vocab = CharacterVocab(SMILES_CHAR_VOCAB)

In [ ]:
ds = TextDataset(df.smiles.values, vocab)

In [ ]:
dl = ds.dataloader(32)

In [ ]:
x,y = next(iter(dl))

In [ ]:
x.shape

torch.Size([32, 66])

In [ ]:
embedding = nn.Embedding(len(vocab.itos), 400, padding_idx=vocab.stoi['pad'])

In [ ]:
out1 = embedding(x)

In [ ]:
out1.shape

torch.Size([32, 66, 400])

In [ ]:
lstm = nn.LSTM(400, 1280, 3, batch_first=True)

In [ ]:
out2 = lstm(out1)

In [ ]:
len(out2)

2

In [ ]:
out2[0].shape

torch.Size([32, 66, 1280])

In [ ]:
len(out2[1])

2

In [ ]:
[i.shape for i in out2[1]]

[torch.Size([3, 32, 1280]), torch.Size([3, 32, 1280])]

In [ ]:
layers.py - common layers
    
models
    agent.py - base agent
    lstm - lstm model + agent
    vae - vae model + agent